### IFN680 Sokoban Assignment


The functions and classes defined in this module will be called by a marker script. 
You should complete the functions and classes according to their specified interfaces.

You are not allowed to change the defined interfaces.
That is, changing the formal parameters of a function will break the 
interface and triggers to a fail for the test of your code.
 

In [3]:
# Imports

import search
import sokoban
import tabooCells
from canGoThereSearch import can_go_there, CanGoThereSearch
from tabooCells import get_taboo_cells_positions
from collections import namedtuple
from SokobanPuzzle import SokobanPuzzle


In [3]:
def my_team():
    '''
    Return the list of the team members of this assignment submission as a list
    of triplet of the form (student_number, first_name, last_name)
    
    '''
    return [(11771984, 'Daniel', 'Skauge'), (123, '...', '...')]

SyntaxError: invalid syntax (<ipython-input-3-efd5212f5cc6>, line 7)

SyntaxError: unexpected EOF while parsing (<ipython-input-14-1a132ed414d0>, line 118)

In [5]:
def check_action_seq(warehouse, action_seq):

    '''
    
    Determine if the sequence of actions listed in 'action_seq' is legal or not.
    
    Important notes:
      - a legal sequence of actions does not necessarily solve the puzzle.
      - an action is legal even if it pushes a box onto a taboo cell.
        
    @param warehouse: a valid Warehouse object

    @param action_seq: a sequence of legal actions.
           For example, ['Left', 'Down', Down','Right', 'Up', 'Down']
           
    @return
        The string 'Failure', if one of the action was not successul.
           For example, if the agent tries to push two boxes at the same time,
                        or push one box into a wall.
        Otherwise, if all actions were successful, return                 
               A string representing the state of the puzzle after applying
               the sequence of actions.  This must be the same string as the
               string returned by the method  Warehouse.__str__()
    '''
    
    ##         "INSERT YOUR CODE HERE"
    
    raise NotImplementedError()
